In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
import cPickle as pickle

In [2]:
%matplotlib inline

In [3]:
pd.set_option('display.max_columns', 500)

#### Clean before model

In [4]:
df = pd.read_pickle('df_1to7499_features.pkl')

In [5]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3,3,93,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269,1,0,1,1,1
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3,3,42,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN,5,0,1,1,1


In [6]:
df.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
4755,7499,225583,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,NE 130TH ST BETWEEN LAKE CITY WAY NE AND 33RD ...,2013-10-01 07:00:00,2013-10-01 07:00:00,2013-10-02 07:00:00,1 days,1,47.744614,-122.294066,"33rd Ave NE, Seattle, WA, USA",9.724967,8.952767,9.586114,9.369915,5.944939,7.978499,4,-4,10,,530330214002,1500000US530330214002,"Block Group 2, Census Tract 214, King County, ...",428400,54553,1,0,1,1,1
4756,7500,210335,EB bike lane between 14th and 16th Ave ...,E UNION ST BETWEEN 15TH AVE AND 16TH AVE ...,2013-07-10 07:00:00,2013-07-12 07:00:00,2013-07-12 07:00:00,2 days,2,47.610378,-122.311513,"16th Ave, Seattle, WA 98122, USA",1.002700,1.897800,1.434032,0.944730,4.473846,2.862636,3,-1,7,55,530330079003,1500000US530330079003,"Block Group 3, Census Tract 79, King County, W...",245000,35437,NaN,NaN,NaN,NaN,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4757 entries, 0 to 4756
Data columns (total 32 columns):
OBJECTID                  4757 non-null int64
WOKEY                     4757 non-null int64
LOCATION                  4757 non-null object
ADDRDESC                  4757 non-null object
INITDT_dt                 4757 non-null datetime64[ns]
FLDSTARTDT_dt             4757 non-null datetime64[ns]
FLDENDDT_dt               4757 non-null datetime64[ns]
DURATION                  4757 non-null timedelta64[ns]
DURATION_td               4757 non-null float64
latitude                  4757 non-null float64
longitude                 4757 non-null float64
address                   4757 non-null object
Seattle_dist              4757 non-null float64
Space_Needle_dist         4757 non-null float64
Pike_Place_dist           4757 non-null float64
Convention_Center_dist    4757 non-null float64
Woodland_Park_dist        4757 non-null float64
Queene_Anne_dist          4757 non-null float64
INIT_Qu

####Remember:  No. of potholes could be more predictive of repair time, so I DO need to include this as a predictor.

In [10]:
df[df['neighborhood_label'] == ''].shape

(172, 32)

In [11]:
df = df[df['neighborhood_label'] != '']

In [12]:
np.any(df.neighborhood_label == '')

False

In [13]:
sum(df.DURATION_td == 0.0)

4

In [14]:
df[df.DURATION_td == 0.0]

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,SND_FEACOD,ST_CODE,SEGMENT_TY,DIVIDED_CO,VEHICLE_US
295,448,338004,Shallow Pothole ...,15TH AVE E BETWEEN E GALER ST AND E GARFIELD S...,2015-03-08 08:00:00,2015-03-09 07:00:00,2015-03-09 07:00:00,23:00:00,0,47.632692,-122.299830,"E Garfield St, Seattle, WA, USA",2.369669,2.458049,2.536548,2.092643,3.527669,2.731130,1,3,3,98,530330062001,1500000US530330062001,"Block Group 1, Census Tract 62, King County, W...",721400,61113,1,0,1,1,1
2718,4256,190832,"698 W MERCER PL, SEATTLE, WA 98119 ...",W MERCER PL BETWEEN 6TH AVE W AND ELLIOTT AVE ...,2013-03-10 08:00:00,2013-03-11 07:00:00,2013-03-11 07:00:00,23:00:00,0,47.625639,-122.367027,"Elliott Ave W, Seattle, WA 98119, USA",2.113811,0.901674,1.598502,1.922310,3.028934,0.922038,1,3,3,45,530330071001,1500000US530330071001,"Block Group 1, Census Tract 71, King County, W...",367000,57656,1,0,1,1,1
4367,6922,338017,Rainier Ave S south of Thistle SB on street ne...,RAINIER AVE S BETWEEN S THISTLE ST AND S CLOVE...,2015-03-08 08:00:00,2015-03-09 07:00:00,2015-03-09 07:00:00,23:00:00,0,47.526137,-122.289320,"S Cloverdale St, Seattle, WA, USA",5.881881,7.096565,6.281264,6.213722,10.286892,8.332554,1,3,3,81,530330117002,1500000US530330117002,"Block Group 2, Census Tract 117, King County, ...",296300,34980,NaN,NaN,NaN,NaN,NaN
4375,6933,338001,Pothole in front of group health between 15th ...,E THOMAS ST BETWEEN 15TH (S LEG) AVE E AND 16T...,2015-03-08 08:00:00,2015-03-09 07:00:00,2015-03-09 07:00:00,23:00:00,0,47.630397,-122.311242,"16th Ave E, Seattle, WA 98112, USA",1.933600,1.903295,2.032375,1.623486,3.295791,2.232374,1,3,3,101,530330064002,1500000US530330064002,"Block Group 2, Census Tract 64, King County, W...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df[df.DURATION_td != 0.0].shape

(4581, 32)

In [16]:
df.shape

(4585, 32)

In [17]:
df = df[df.DURATION_td != 0.0]

In [19]:
df.shape

(4581, 32)

In [20]:
df[pd.isnull(df.Median_Value)].shape

(285, 32)

In [22]:
df[pd.isnull(df.Margin_of_Error)].shape

(285, 32)

In [23]:
df[pd.isnull(df.Margin_of_Error)].shape[0]/float(df.shape[0])

0.06221349050425671

####6% of Median Value are NaN.  285/4581.  Ignore and remove for now.

In [24]:
df = df[np.isfinite(df['Median_Value'])]

In [25]:
df.shape

(4296, 32)

####Now it's time to deal with the street features.  I also want to use this notebook to develop some EDA of the duration variable for tomorrow.